In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## Package Load

In [ ]:
!pip install bloxs
from bloxs import B

import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import GlobalMaxPooling1D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.layers import Concatenate, LSTM, GRU
from tensorflow.keras.layers import Bidirectional, Multiply

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import roc_auc_score

from sklearn.model_selection import KFold, GroupKFold

from sklearn.model_selection import train_test_split

## 1. Train Data

In [ ]:
tr = pd.read_csv("/kaggle/input/tabular-playground-series-apr-2022/train.csv")
tr

In [ ]:
display(tr.info())

display(tr.describe())

display(tr.nunique())

In [ ]:
display(B(tr['subject'].nunique(), 'Subjects'),
B(tr['sequence'].nunique(), 'Sequences'),
B(tr['step'].nunique(), 'Steps'))

In [ ]:
sequences = [0, 1, 2, 8364, 15404]
figure, axes = plt.subplots(13, len(sequences), sharex=True, figsize=(16, 16), facecolor='#f6f5f5')
colors = ['#7A5197', '#BB5098', '#5344A9', '#F5C63C', '#F47F6B']

for i, sequence in enumerate(sequences):
    for sensor in range(13):
        sensor_name = f"sensor_{sensor:02d}"
        ax = plt.subplot(13, len(sequences), sensor * len(sequences) + i + 1)
        for s in ["top","right"]:
                ax.spines[s].set_visible(False)
        ax.set_facecolor('#f6f5f5')
        plt.plot(range(60), tr[tr.sequence == sequence][sensor_name],
                color=colors[i]);        #plt.rcParams['axes.prop_cycle'].by_key()['color'][i % 10])
        if sensor == 0: plt.title(f"Sequence {sequence}");
        if sequence == sequences[0]: plt.ylabel(sensor_name);
figure.tight_layout(w_pad=0.1)
plt.suptitle('Selected Time Series', y=1.02, fontweight='bold');
plt.show();

## 2. Train Label

In [ ]:
tr_label = pd.read_csv("/kaggle/input/tabular-playground-series-apr-2022/train_labels.csv")
tr_label

In [ ]:
display(tr_label.info())
display(tr_label.state.value_counts())

In [ ]:
colors = ['#7A5197', '#BB5098', '#5344A9', '#F5C63C', '#F47F6B']

plt.subplots(figsize=(25, 10), facecolor='#f6f5f5')
plt.pie(tr_label['state'].value_counts(), startangle=90, wedgeprops={'width':0.3}, colors=['#F5C63C', '#7A5197'] )
plt.title('Target Balance Pie Chart', loc='center', fontsize=24, color='#7A5197', fontweight='bold');
plt.text(0, 0, f"{tr_label['state'].value_counts()[0] / tr_label['state'].count() * 100:.2f}%", ha='center', va='center', fontweight='bold', fontsize=42, color='#7A5197');
plt.legend(tr_label['state'].value_counts().index, ncol=2, facecolor='#f6f5f5', edgecolor='#f6f5f5', loc='lower center', fontsize=16);
plt.show();

## 3. Test Data

In [ ]:
te = pd.read_csv("/kaggle/input/tabular-playground-series-apr-2022/test.csv")
te

In [ ]:
display(te.info())

display(te.describe())

display(te.nunique())

In [ ]:
display(B(te['subject'].nunique(), 'Subjects'),
B(te['sequence'].nunique(), 'Sequences'),
B(te['step'].nunique(), 'Steps'))

## 4. GRU by Tensorflow

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-apr-2022/train.csv')
train_labels = pd.read_csv('../input/tabular-playground-series-apr-2022/train_labels.csv')
test_df = pd.read_csv('../input/tabular-playground-series-apr-2022/test.csv')
ssub = pd.read_csv('../input/tabular-playground-series-apr-2022/sample_submission.csv')

In [ ]:
# 센서 변수들 시계열 처리
features = train_df.columns.tolist()[3:]

def prep(df):
    for feature in features:
        df[feature + '_lag1'] = df.groupby('sequence')[feature].shift(1)
        df.fillna(0, inplace=True)
        df[feature + '_diff1'] = df[feature] - df[feature + '_lag1']    

prep(train_df)
prep(test_df)

# 변수들 정규화
features = train_df.columns.tolist()[3:]
sc = StandardScaler()
train_df[features] = sc.fit_transform(train_df[features])
test_df[features] = sc.transform(test_df[features])

# 필요 데이터 추출
groups = train_df["sequence"]
labels = train_labels["state"]

# 학습에 사용하지 않는 변수 제거 및 데이터 차원 변환
train_df = train_df.drop(["sequence", "subject", "step"], axis=1).values
train_df = train_df.reshape(-1, 60, train_df.shape[-1])

test_df = test_df.drop(["sequence", "subject", "step"], axis=1).values
test_df = test_df.reshape(-1, 60, test_df.shape[-1])

In [ ]:
# 모델 구축    
def gru_model():
    
    x_input = Input(shape=(train_df.shape[-2:]))
    
    x1 = Bidirectional(GRU(units=512, return_sequences=True))(x_input)
    x2 = Bidirectional(GRU(units=256, return_sequences=True))(x1)
    x3 = GlobalMaxPooling1D()(x2)
    x4 = Dense(units=128, activation='selu')(x3)
    x_output = Dense(1, activation='sigmoid')(x4)

    model = Model(inputs=x_input, outputs=x_output, name='gru')
    
    return model

model = gru_model()

In [ ]:
BATCH_SIZE = 512
VERBOSE = True
predictions, scores = [], []


X_train, X_val, y_train, y_val = train_test_split(train_df,labels)

model = gru_model()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics='AUC')
save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')

model.fit(X_train, y_train, 
          validation_data=(X_val, y_val), 
          epochs=15,
          verbose=VERBOSE,
          batch_size=BATCH_SIZE)


y_pred = model.predict(X_val, batch_size=BATCH_SIZE).squeeze()
score = roc_auc_score(y_val, y_pred)
scores.append(score)

predictions.append(model.predict(test_df, batch_size=BATCH_SIZE).squeeze())

In [ ]:
predictions.append(model.predict(test_df, batch_size=BATCH_SIZE).squeeze())

In [ ]:
y_pred = model.predict(X_val, batch_size=BATCH_SIZE)
score = roc_auc_score(y_val, y_pred)
scores.append(score)
scores

In [ ]:
predictions

In [ ]:
ssub["state"] = sum(predictions)
ssub.to_csv('submition.csv', index=False)
ssub.head(3)

Reference

<a href="https://www.kaggle.com/code/kartushovdanil/top-1-tps-apr-22-eda-lstm#4.-Explore-Data-Analisyse">@Torch me</a>